<a href="https://colab.research.google.com/github/cabbi120101/TIL/blob/master/ML/2%EC%B0%A8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%EB%A6%AC%EB%B7%B0%EB%8A%94%EB%82%98%EC%98%A4%EB%8A%94%EB%8D%B03%EA%B3%B3%EB%A7%8C%EB%90%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install selenium

In [2]:
import sys
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [3]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
# options = webdriver.ChromeOptions()
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome('chromedriver',options=options)

# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("place")

# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['신림 카페']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

##### 0
#### 디자이너리카페
no review in extract
환하고 예쁨. ㅁ자형 구조라 가운데 트인 부분이 기분을 상쾌하게 해줌. 커피맛은 잘 모르므로 이 부분에 대한 언급은 패스. 앉아서 노트북하고 공부하기 좋은 테이블, 의자였음. 재방문의사 있음./5
매장 자체는 예쁜데 맛은 없어서 항상 재방문이 망설여짐/3
넘 예쁘고 음료, 디저트도 맛나용/5
장소가 주는 힙함. 아늑함. /4
루프탑 풍경도 괜찮고 모던 인테리어치고 편안한 느낌. 햇빛이 잘 드는 지 화이트톤이라 그런건지 구름낀 날씨에도 눈부시고 맑아서 책 보기에 좋았음 친구들이랑 수다떨러 오고 싶은 곳. 근데 커피가 진짜 맛없음/3
주말에 자리 진짜 없음.. 햇볕 강한 날 가면 직사광선에 온 몸 타는 기분을 느낄 수 있음 그리고 루프탑과 연결된 중정형?? 이라 그런지 더 시끄러운 것 같아어요. 도떼기시장 저리가라/3
이쁜데 맛은 없음/3
시설이랑 이런건 좋은데... 서비스는 조금 아쉽네요~! 조금더 좋게 해주셨으면 좋겠어요~/4
/2
/5
분위기에 취한다/4
자리값 커피는 모르겠다/3
이 동네에 이런 카페가...? 와우! 믿기지않는다! 깔끔하고 넓은 내부, 독특한 인테리어에 루프탑까지 완벽- 다만, 커피맛이 좀 별로다. 여기 올거면 아아메 먹던지 차라리 병음료 추천 /4
음료 진짜 맛없어요 자리 없는건 그러려니 하는데 시럽덩어리에 물 푼 맛임/1
/4
카페 인테리어는 정말 이쁘고 쾌적해서 자주오고 싶을것 같아요   그런데 커피양이 좀 적은것 같은건 기분탓이려나요 ㅠㅠ/3
아메리카노 시켰는데 원두 다타서 두입먹고 다버림 뭐임이거 탄맛 에바잖어 ㅇ이거 다먹으면 암걸릴듯/1
커피맛은 평범? 불만갖거나 감탄할 맛이 아니었고 디저트류도 지극히 평범 케잌은 떼오는것 같고 토스트는 빵 생크림 사과잼 모두가 평범합니다 의자도 딱딱하고 불편하고요 그렇지만 이곳을 먹으러 오는 곳이 아니라 널찍한 공간을 즐기러 오는 곳으로 생각한다면 신림에서 가장 만족스러운 공간일것입니다!/4
카페는 예쁘지만 찾아가는 길 

NoSuchElementException: ignored

In [6]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
# options = webdriver.ChromeOptions()
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome('chromedriver',options=options)

# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("카페")

# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['신림 카페']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

##### 0
#### 디자이너리카페
no review in extract
환하고 예쁨. ㅁ자형 구조라 가운데 트인 부분이 기분을 상쾌하게 해줌. 커피맛은 잘 모르므로 이 부분에 대한 언급은 패스. 앉아서 노트북하고 공부하기 좋은 테이블, 의자였음. 재방문의사 있음./5
매장 자체는 예쁜데 맛은 없어서 항상 재방문이 망설여짐/3
넘 예쁘고 음료, 디저트도 맛나용/5
장소가 주는 힙함. 아늑함. /4
루프탑 풍경도 괜찮고 모던 인테리어치고 편안한 느낌. 햇빛이 잘 드는 지 화이트톤이라 그런건지 구름낀 날씨에도 눈부시고 맑아서 책 보기에 좋았음 친구들이랑 수다떨러 오고 싶은 곳. 근데 커피가 진짜 맛없음/3
주말에 자리 진짜 없음.. 햇볕 강한 날 가면 직사광선에 온 몸 타는 기분을 느낄 수 있음 그리고 루프탑과 연결된 중정형?? 이라 그런지 더 시끄러운 것 같아어요. 도떼기시장 저리가라/3
이쁜데 맛은 없음/3
시설이랑 이런건 좋은데... 서비스는 조금 아쉽네요~! 조금더 좋게 해주셨으면 좋겠어요~/4
/2
/5
분위기에 취한다/4
자리값 커피는 모르겠다/3
이 동네에 이런 카페가...? 와우! 믿기지않는다! 깔끔하고 넓은 내부, 독특한 인테리어에 루프탑까지 완벽- 다만, 커피맛이 좀 별로다. 여기 올거면 아아메 먹던지 차라리 병음료 추천 /4
음료 진짜 맛없어요 자리 없는건 그러려니 하는데 시럽덩어리에 물 푼 맛임/1
/4
카페 인테리어는 정말 이쁘고 쾌적해서 자주오고 싶을것 같아요   그런데 커피양이 좀 적은것 같은건 기분탓이려나요 ㅠㅠ/3
아메리카노 시켰는데 원두 다타서 두입먹고 다버림 뭐임이거 탄맛 에바잖어 ㅇ이거 다먹으면 암걸릴듯/1
커피맛은 평범? 불만갖거나 감탄할 맛이 아니었고 디저트류도 지극히 평범 케잌은 떼오는것 같고 토스트는 빵 생크림 사과잼 모두가 평범합니다 의자도 딱딱하고 불편하고요 그렇지만 이곳을 먹으러 오는 곳이 아니라 널찍한 공간을 즐기러 오는 곳으로 생각한다면 신림에서 가장 만족스러운 공간일것입니다!/4
카페는 예쁘지만 찾아가는 길 

NoSuchElementException: ignored

In [ ]:
from bs4 import BeautifulSoup
import requests
import json            #json import하기

custom_header = {
    'referer' : 'https://www.mangoplate.com/',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36' }


def get_restaurants(name) :
    # 검색어 name이 들어왔을 때 검색 결과로 나타나는 식당들을 리스트에 담아 반환
    
    restuarant_list = []
    
    url = "https://www.mangoplate.com/search/" + name
    
    req = requests.get(url, headers = custom_header)
    
    soup = BeautifulSoup(req.text, "html.parser")
    
    div = soup.find_all("div",class_="info")
    
    for re in div:
        href = re.find("a")['href']
        title = re.find("h2",class_="title").get_text().replace("\n","").replace("  ","")
        restuarant_list.append([href,title])
    
    return restuarant_list
    

def main() :
    name = input()
    
    restuarant_list = get_restaurants(name)
    
    print(restuarant_list)

if __name__ == "__main__" :
    main()

신림
[['/restaurants/CbdJqQErxKsX', '빠사삭'], ['/restaurants/HoWked2wivcI', '중화요리팔공'], ['/restaurants/DuYISQu8X9', '서울갈비'], ['/restaurants/A3itRxUfBs', '춘천골참숯불닭갈비'], ['/restaurants/N6zsknPkBG5l', '동학'], ['/restaurants/P_0ncHUU8dv6', '소미당'], ['/restaurants/bgbs23fGlnTp', '고모네정육식당'], ['/restaurants/cWfAVTOfVdLb', '발루토'], ['/restaurants/hX0tke6m83', '아리차이'], ['/restaurants/pS4G5e128O', '오첨지'], ['/restaurants/IY4bulaHdHWG', '호텔 베라노 (휴업중)'], ['/restaurants/-EOPKh3PhlzZ', '구루메키친'], ['/restaurants/hdeyUljHK5o6', '막불감동'], ['/restaurants/-C1tadV-lA', '다케'], ['/restaurants/RW7UpZAkatp2', '신림동피자집'], ['/restaurants/s5pHAbDq_71M', '마뇨떡볶이'], ['/restaurants/0ApBtrtmrK', '호남집'], ['/restaurants/LyypO1XY8u', '나폴리네'], ['/restaurants/ZknfMajVJu', '가네샤'], ['/restaurants/W7QiHQonSe', '삼촌네']]


In [ ]:
main()

카페
[['/restaurants/Fy2Z5kqgi-7b', '따빠디또'], ['/restaurants/aeO_cG35KO', '패스트리부티크(서울신라호텔)'], ['/restaurants/vcheyGIxPJ9D', '껠끄쇼즈'], ['/restaurants/0my2hMzHlOYD', '당도'], ['/restaurants/8GRQ6g5tH8GB', '플랜트(2호점)'], ['/restaurants/RHt4Mtrl4zHp', '듁스커피'], ['/restaurants/-czYKuW77F_5', '파네트크루아상팩토리(광화문점)'], ['/restaurants/G91eF6EI4DPc', '스티키리키'], ['/restaurants/XRoMziImmYCC', '뉴욕택시디저트'], ['/restaurants/SaCJTvz6fPV8', '티크닉'], ['/restaurants/faOJKeUy00Yt', '쎄쎄종'], ['/restaurants/QxTRZZv3Dl', '삐아프'], ['/restaurants/dPMQH4nlqZXc', '콰이어트 크림티'], ['/restaurants/KyJsLQGd64', '브라운브레드(도곡점)'], ['/restaurants/8mhsdtiBg0', '아티장베이커스(한남점)'], ['/restaurants/WjvK9ngVzlAg', '성심당(대전컨벤션센터점)'], ['/restaurants/DM2xUaLRreaa', '꼼다비뛰드'], ['/restaurants/JlEOAtsbIt', '포비(디타워점)'], ['/restaurants/q-ImZ-DGRhlY', '소울브레드'], ['/restaurants/jbXHGq3QwY', '폴앤폴리나(광화문점)']]


In [ ]:
from bs4 import BeautifulSoup
import requests
import json            #json import하기

custom_header = {
    'referer' : 'https://www.mangoplate.com/',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36' }

def get_reviews(code) :
    comments = []
    
    url = f"https://stage.mangoplate.com/api/v5{code}/reviews.json?language=kor&device_uuid=V3QHS15862342340433605ldDed&device_type=web&start_index=0&request_count=5&sort_by=2"
    req = requests.get(url, headers = custom_header)
    
    if req.status_code == requests.codes.ok:
        result = []
        print("접속 성공")
        reviews = json.loads(req.text)
        for i in range(5):
            result.append(reviews[i]["comment"]["comment"][:20].replace("\n",""))
    else:
        print("Error code")
    # req에 데이터를 불러온 결과가 저장되어 있습니다.
    # JSON으로 저장된 데이터에서 댓글을 추출하여 comments에 저장하고 반환하세요.
    return result
    
    

def get_restaurants(name) :
    url = f"https://www.mangoplate.com/search/{name}"
    req = requests.get(url, headers = custom_header)
    soup = BeautifulSoup(req.text, "html.parser")
    
    # soup에는 특정 키워드로 검색한 결과의 HTML이 담겨 있습니다.
    result = []
    
    restaurants = soup.find_all("div",class_="info")
    
    # 특정 키워드와 관련된 음식점의 이름과 href를 튜플로 저장하고,
    for res in restaurants:
        name = res.find("h2",class_="title").get_text().replace("\n","").replace("  ","")
        a_tag = res.find("a")["href"]
        result.append((name,a_tag))
    # 이름과 href를 담은 튜플들이 담긴 리스트를 반환하세요.
    return result
    
def main() :
    name = input("검색어를 입력하세요 : ")
    
    restuarant_list = get_restaurants(name)
    
    for r in restuarant_list :
        print(r[0])
        print(get_reviews(r[1]))
        print("="*30)
        print("\n"*2)

if __name__ == "__main__" :
    main()

검색어를 입력하세요 : 신림
빠사삭
접속 성공
['튀김 맛집.. 그동안 먹었던 튀김과는', '최고의튀김... 빠사삭으로 오리까지 ', '너무 너무 가고싶어서 벼르다가 오늘 ', '관악구 뜨기전에 녹두 맛집 다 뿌숴야', '떡볶이 먹을 때 튀김 안먹는 사람인데']



중화요리팔공
접속 성공
['짜장면 볶음밥 탕수육 먹었는데 짜장 ', '짜장면과 볶음밥 주문짜장은 양파가', '부천에서 종종 가는 중식당에서 추천을', '맛있어요...', ' 짜장면이 6천원인데 간짜장이 따로 ']



서울갈비
접속 성공
['고기 특징없고 소스도 제품맛 물씬.', '우삼겹, 된장찌개, 공기밥신림의 ', '우삼겹맛집맛있다! 이미 양념이 ', '종종들리는 맛집딱 땡기는 달콤소스+', '맛있어요 ㅎㅎ']



춘천골참숯불닭갈비
접속 성공
['닭갈비 4/5 : 적당히 괜찮음. 부', '맛있게 잘 먹고왔어요. 막국수랑 된장', '불친절하고 맛별로', '간판없는 닭갈비집!! 동네에 사는 아', '토속적이고 깊은 맛 좋네요']



동학
접속 성공
['아 컨셉최고...  여기 구조장난없네', '해물파전을 먹었는데 그냥 그랬어요. ', '영화에 나오는 지하세계인줄 알았다. ', '#냠냠여름하면 장마 장마하면 막걸', '막걸리랑 전을 입에도 안 대는 내가 ']



소미당
접속 성공
['일단 가게는 작다 6테이블 하나는 1', '연어덮밥이 말도 안되게 맛있는 곳', '믹스가츠동 7500 싼것같다. 맛있다', '맛있어요!!연어 엄청 신선하고, ', '사진은 사케동(9500원). 밥알은']



고모네정육식당
접속 성공
['8천원의 행복이랄까.#고모네정육식', '생각없이 방문했는데 이 가격에 이 정', 'It’s totally worth t', '고모네 정육식당(고모네 육회맛집) ★', '육회비빔밥 1만원, 육회냉면 8천원 ']



발루토
접속 성공
['스콘의 기본기가 되어있는 집택배로도', '녹두 카페 추천해달라고 하면 할리스.', '스콘 맛집이라고 먼 길을 찾아가면 실', '넓은 테이블 간격,혼

In [ ]:
import nowon

ModuleNotFoundError: ignored

In [ ]:
rating_list = []
userid_list = []
itemid_list = []
timestamp_list = []
comment_list = []

driver = webdriver.Chrome('chromedriver',options=options)

for k, row in nowon.iterrows():
    url = nowon["kakao_url"][k]
    print(k,"번째 크롤링 중 : ",nowon["상호명"][k] , sep=" ")
    driver.get(url)

    time.sleep(2)

try:
    rating_num = driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_sorting > a > span.color_b').text
    rating_num = int(rating_num)
    print(rating_num)
    
    # 별점 개수로 페이지를 넘기는 기준을 정하기로 함 / 한 페이지 당 최대 리뷰는 5개!
    if rating_num > 0 and rating_num <= 5:
        p_num = 1
        print(p_num)
        
        time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        user = driver.find_elements_by_xpath("//a[@class='link_user']")
        review_lists = soup.select('.list_evaluation > li')
        time.sleep(3)
        for j in range(0, len(user)):
            userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
            itemid_list.append(nowon["상호명"][k])
            timestamp_list.append(soup.select('.time_write')[j].text)
            time.sleep(3)
        if len(review_lists) != 0:
            for j, review in enumerate(review_lists):
                comment = review.select('.txt_comment > span') 
                rating = review.select('.grade_star > em') # 별점
                        
                if len(comment) != 0:
                    if len(rating) != 0 :
                        comment_list.append(comment[0].text) 
                        rating_list.append(rating[0].text)
                        time.sleep(3)
                    else:
                        comment_list.append(comment[0].text) 
                        rating_list.append('0')
        else:
            print('리뷰가 없습니다.')
            time.sleep(3)    
                
                
    # 2 페이지
    elif rating_num >= 6 and rating_num <= 10:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        user = driver.find_elements_by_xpath("//a[@class='link_user']")
        review_lists = soup.select('.list_evaluation > li')
        time.sleep(3)
        for j in range(0, len(user)):
            userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
            itemid_list.append(nowon["상호명"][k])
            timestamp_list.append(soup.select('.time_write')[j].text)
            time.sleep(3)
        if len(review_lists) != 0:
            for j, review in enumerate(review_lists):
                comment = review.select('.txt_comment > span') 
                rating = review.select('.grade_star > em') # 별점
                        
                if len(comment) != 0:
                    if len(rating) != 0:
                        comment_list.append(comment[0].text) 
                        rating_list.append(rating[0].text)
                        time.sleep(3)
                    else:
                        comment_list.append(comment[0].text) 
                        rating_list.append('0')
        else:
            print('리뷰가 없습니다.')
            time.sleep(3)
                
        driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a').click()
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        user = driver.find_elements_by_xpath("//a[@class='link_user']")
        review_lists = soup.select('.list_evaluation > li')
        time.sleep(3)
        
        for j in range(0, len(user)):
            userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
            itemid_list.append(nowon["상호명"][k])
            timestamp_list.append(soup.select('.time_write')[j].text)
            time.sleep(3)
        if len(review_lists) != 0:
            for j, review in enumerate(review_lists):
                comment = review.select('.txt_comment > span') 
                rating = review.select('.grade_star > em') # 별점
                        
                if len(comment) != 0:
                    if len(rating) != 0:
                        comment_list.append(comment[0].text) 
                        rating_list.append(rating[0].text)
                        time.sleep(3)
                    else:
                        comment_list.append(comment[0].text) 
                        rating_list.append('0')
        else:
            print('리뷰가 없습니다.')
            time.sleep(3)
                
    # 3페이지 이하         
    elif rating_num >= 11 and rating_num <= 15:
        p_num = 3
        print(p_num)
                
        for i in range(1,p_num+1):
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            user = driver.find_elements_by_xpath("//a[@class='link_user']")
            review_lists = soup.select('.list_evaluation > li')
            time.sleep(3)
            for j in range(0, len(user)):
                userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                itemid_list.append(nowon["상호명"][k])
                timestamp_list.append(soup.select('.time_write')[j].text)
            
        # 카카오맵의 개인 user별 url의 규칙은 다음과 같습니다.
                time.sleep(3)
            if len(review_lists) != 0:
                for j, review in enumerate(review_lists):
                    comment = review.select('.txt_comment > span') 
                    rating = review.selec('.grade_star > em')
                        
                    if len(comment) != 0:
                        if len(rating) != 0:
                            comment_list.append(comment[0].text) 
                            rating_list.append(rating[0].text)
                            time.sleep(3)
                        else:
                            comment_list.append(comment[0].text) 
                            rating_list.append('0')
            else:
                print('리뷰가 없습니다.')
            try: # 마지막 페이지 크롤링을 위해서 try사용
                driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(i) +']').click() # 페이지별로 실행하고
            except:
                continue
            time.sleep(2)
            
            
    # 4페이지     
    elif rating_num >= 16 and rating_num <= 20:
        p_num = 4
        print(p_num)
        
        for i in range(1,p_num+1):
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            user = driver.find_elements_by_xpath("//a[@class='link_user']")
            review_lists = soup.select('.list_evaluation > li')
            time.sleep(2)
            for j in range(0, len(user)):
                userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                itemid_list.append(nowon["상호명"][k])
                timestamp_list.append(soup.select('.time_write')[j].text)
        # 카카오맵의 개인 user별 url의 규칙은 다음과 같습니다.
            time.sleep(2)
            
            if len(review_lists) != 0:
                for j, review in enumerate(review_lists):
                    comment = review.select('.txt_comment > span') 
                    rating = review.select('.grade_star > em')
                        
                    if len(comment) != 0:
                        if len(rating) != 0:
                            comment_list.append(comment[0].text) 
                            rating_list.append(rating[0].text)
                            time.sleep(3)
                        else:
                            comment_list.append(comment[0].text) 
                            rating_list.append('0')
            else:
                print('리뷰가 없습니다.')
            try: # 마지막 페이지 크롤링을 위해서 try사용
                driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(i) +']').click() # 페이지별로 실행하고
            except:
                continue
            time.sleep(2)
        
                    
        
                    
    # 5페이지 이상           
    elif rating_num >= 21:
        p_num = int(math.ceil(rating_num / 5))
        print(p_num)
        
        for i in range(1, 7):
            p_num -= 1
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            user = driver.find_elements_by_xpath("//a[@class='link_user']")
            review_lists = soup.select('.list_evaluation > li')
            time.sleep(2)
            for j in range(0, len(user)):
                userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                itemid_list.append(nowon["상호명"][k])
                timestamp_list.append(soup.select('.time_write')[j].text)
            # 카카오맵의 개인 user별 url의 규칙은 다음과 같습니다.
                time.sleep(3)
            
            
                # 리뷰 리스트가 0이 아니면
            if len(review_lists) != 0:
                for j, review in enumerate(review_lists):
                    comment = review.select('.txt_comment > span') 
                    rating = review.select('.grade_star > em') # 별점
                        
                    if len(comment) != 0:
                        if len(rating) != 0:
                            comment_list.append(comment[0].text) 
                            rating_list.append(rating[0].text)
                            time.sleep(3)
                        else:
                            comment_list.append(comment[0].text) 
                            rating_list.append('0')
            else:
                print('리뷰가 없습니다.')
            try: # 마지막 페이지 크롤링을 위해서 try사용
                driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(i) +']').click() # 페이지별로 실행하고
            except:
                continue
            time.sleep(2)
            print(p_num)
            pp_num = p_num
        while p_num <= pp_num and p_num >= 0:
            for a in range(2,7) :
                p_num -= 1
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                user = driver.find_elements_by_xpath("//a[@class='link_user']")
                time.sleep(2)
                for j in range(0,len(user)):
                    userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                    itemid_list.append(nowon["상호명"][k])
                    timestamp_list.append(soup.select('.time_write')[j].text)
                    time.sleep(2)
                if len(review_lists) != 0:
                    for i, review in enumerate(review_lists):
                        comment = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        if len(comment) != 0:
                            if len(rating) != 0:
                                comment_list.append(comment[0].text) 
                                rating_list.append(rating[0].text)
                                time.sleep(3)
                            else:
                                comment_list.append(comment[0].text) 
                                rating_list.append('0')
                else:
                    print('no review in extract')
                try:
                    driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(a) +']').click() # 페이지별로 실행하고
                except:
                    continue
                time.sleep(3)    
            
            
                            
                                       

except (NoSuchElementException, ElementNotInteractableException):
    print("리뷰가 없습니다")
    continue

NameError: ignored